In [3]:
import pandas as pd
import tensorflow as tf
import os
from keras.utils import load_img

In [9]:
image = []
image_dir = 'image' 
for filename in os.listdir(image_dir):
    file_path = os.path.join(image_dir, filename)  
    img = load_img(file_path, target_size=(224, 224))
    image.append(img)
    
print(image)

[<PIL.Image.Image image mode=RGB size=224x224 at 0x1FCABD12F10>, <PIL.Image.Image image mode=RGB size=224x224 at 0x1FCABED64C0>, <PIL.Image.Image image mode=RGB size=224x224 at 0x1FCB54DE070>, <PIL.Image.Image image mode=RGB size=224x224 at 0x1FCABEADA30>, <PIL.Image.Image image mode=RGB size=224x224 at 0x1FCABEADA00>]
